# 密集连接网络 (Dense/Fully Connected Networks)

## 核心概念

密集连接网络（也称全连接网络）是最基础的神经网络结构，由Dense层堆叠而成。每个神经元都与前一层的所有神经元连接。

### 核心特性

1. **全连接性** (Fully Connected)
   - 每个神经元与上一层所有神经元相连
   - 参数量 = `input_size × output_size + output_size`
   - 可以学习任意复杂的非线性映射

2. **层级特征提取**
   - 浅层学习简单特征组合
   - 深层学习抽象特征表示
   - 逐层提取高阶特征

3. **通用逼近能力**
   - 理论上可以逼近任意连续函数
   - 需要足够的隐藏层和神经元
   - 实践中需要合理设计防止过拟合

### 适用场景

- **结构化表格数据**: 特征工程后的向量数据
- **分类任务**: 二分类、多分类、多标签分类
- **回归任务**: 单变量回归、多变量回归
- **网络末端**: 作为CNN、RNN等网络的分类/回归头

---

## 1. 环境准备

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist

from sklearn.datasets import make_classification, make_multilabel_classification, make_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, mean_squared_error, r2_score

print(f"TensorFlow版本: {tf.__version__}")
print(f"GPU可用: {tf.config.list_physical_devices('GPU')}")

# 设置随机种子
np.random.seed(42)
tf.random.set_seed(42)

---

## 2. 二分类问题 (Binary Classification)

### 关键配置
- **输出层**: 1个神经元 + sigmoid激活
- **损失函数**: `binary_crossentropy`
- **评估指标**: accuracy, precision, recall, AUC

### 典型应用
- 垃圾邮件检测
- 欺诈检测
- 疾病诊断（是/否）
- 情感分析（正面/负面）

In [ ]:
# 生成二分类数据集
X_binary, y_binary = make_classification(
    n_samples=10000,
    n_features=20,
    n_informative=15,
    n_redundant=5,
    n_classes=2,
    random_state=42
)

# 数据划分
X_train_bin, X_test_bin, y_train_bin, y_test_bin = train_test_split(
    X_binary, y_binary, test_size=0.2, random_state=42, stratify=y_binary
)

# 标准化
scaler_bin = StandardScaler()
X_train_bin = scaler_bin.fit_transform(X_train_bin)
X_test_bin = scaler_bin.transform(X_test_bin)

print(f"训练集形状: {X_train_bin.shape}")
print(f"测试集形状: {X_test_bin.shape}")
print(f"类别分布: {np.bincount(y_train_bin)}")

In [ ]:
def build_binary_classifier(input_dim, hidden_layers=[64, 32], dropout_rate=0.3):
    """
    构建二分类模型
    
    参数:
        input_dim: 输入特征维度
        hidden_layers: 隐藏层神经元数量列表
        dropout_rate: Dropout比例
    
    返回:
        编译后的模型
    """
    model = models.Sequential(name='Binary_Classifier')
    
    # 输入层
    model.add(layers.Input(shape=(input_dim,)))
    
    # 隐藏层
    for i, units in enumerate(hidden_layers):
        model.add(layers.Dense(units, activation='relu', name=f'hidden_{i+1}'))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(dropout_rate))
    
    # 输出层：1个神经元 + sigmoid激活
    # sigmoid将输出映射到(0, 1)，表示属于正类的概率
    model.add(layers.Dense(1, activation='sigmoid', name='output'))
    
    return model

In [ ]:
# 构建并编译二分类模型
model_binary = build_binary_classifier(input_dim=X_train_bin.shape[1])

model_binary.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',  # 二分类交叉熵损失
    metrics=['accuracy', 
             keras.metrics.Precision(name='precision'),
             keras.metrics.Recall(name='recall'),
             keras.metrics.AUC(name='auc')]
)

model_binary.summary()

In [ ]:
# 训练模型（测试用简化参数）
history_binary = model_binary.fit(
    X_train_bin, y_train_bin,
    batch_size=128,
    epochs=30,  # 测试时用2个epoch，实际训练建议30-50
    validation_split=0.2,
    verbose=1
)

In [ ]:
# 评估二分类模型
results = model_binary.evaluate(X_test_bin, y_test_bin, verbose=0)
print("\n" + "="*50)
print("二分类模型测试结果:")
for name, value in zip(model_binary.metrics_names, results):
    print(f"{name}: {value:.4f}")

# 预测
y_pred_prob = model_binary.predict(X_test_bin, verbose=0)
y_pred_bin = (y_pred_prob > 0.5).astype(int).flatten()

# 混淆矩阵
cm = confusion_matrix(y_test_bin, y_pred_bin)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('二分类混淆矩阵')
plt.ylabel('真实标签')
plt.xlabel('预测标签')
plt.show()

print("\n分类报告:")
print(classification_report(y_test_bin, y_pred_bin))

---

## 3. 单标签多分类问题 (Multi-class Classification)

### 关键配置
- **输出层**: N个神经元 (N=类别数) + softmax激活
- **损失函数**: 
  - 标签是one-hot编码: `categorical_crossentropy`
  - 标签是整数编码: `sparse_categorical_crossentropy`
- **评估指标**: accuracy, top-k accuracy

### 典型应用
- 手写数字识别
- 文档分类
- 图像分类
- 物种识别

In [ ]:
# 使用MNIST数据集作为多分类示例
(X_train_multi, y_train_multi), (X_test_multi, y_test_multi) = mnist.load_data()

# 展平图像：28x28 → 784
X_train_multi = X_train_multi.reshape(-1, 784).astype('float32') / 255.0
X_test_multi = X_test_multi.reshape(-1, 784).astype('float32') / 255.0

# 标签one-hot编码
num_classes = 10
y_train_multi_cat = keras.utils.to_categorical(y_train_multi, num_classes)
y_test_multi_cat = keras.utils.to_categorical(y_test_multi, num_classes)

print(f"训练集形状: {X_train_multi.shape}")
print(f"标签形状(one-hot): {y_train_multi_cat.shape}")
print(f"类别数: {num_classes}")

In [ ]:
def build_multiclass_classifier(input_dim, num_classes, hidden_layers=[256, 128, 64]):
    """
    构建多分类模型
    
    参数:
        input_dim: 输入特征维度
        num_classes: 类别数量
        hidden_layers: 隐藏层神经元数量列表
    
    返回:
        编译后的模型
    """
    model = models.Sequential(name='Multiclass_Classifier')
    
    model.add(layers.Input(shape=(input_dim,)))
    
    # 隐藏层
    for i, units in enumerate(hidden_layers):
        model.add(layers.Dense(units, activation='relu', name=f'hidden_{i+1}'))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(0.3))
    
    # 输出层：N个神经元 + softmax激活
    # softmax确保输出和为1，每个值表示对应类别的概率
    model.add(layers.Dense(num_classes, activation='softmax', name='output'))
    
    return model

In [ ]:
# 构建并编译多分类模型
model_multiclass = build_multiclass_classifier(
    input_dim=784,
    num_classes=10
)

model_multiclass.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',  # one-hot编码用categorical
    metrics=['accuracy', keras.metrics.TopKCategoricalAccuracy(k=3, name='top3_acc')]
)

model_multiclass.summary()

In [ ]:
# 训练模型
history_multiclass = model_multiclass.fit(
    X_train_multi, y_train_multi_cat,
    batch_size=256,
    epochs=30,  # 测试时用2个epoch
    validation_split=0.1,
    verbose=1
)

In [ ]:
# 评估多分类模型
results = model_multiclass.evaluate(X_test_multi, y_test_multi_cat, verbose=0)
print("\n" + "="*50)
print("多分类模型测试结果:")
for name, value in zip(model_multiclass.metrics_names, results):
    print(f"{name}: {value:.4f}")

# 预测并可视化
y_pred_multi = model_multiclass.predict(X_test_multi[:16], verbose=0)
y_pred_classes = np.argmax(y_pred_multi, axis=1)

plt.figure(figsize=(12, 6))
for i in range(16):
    plt.subplot(2, 8, i+1)
    plt.imshow(X_test_multi[i].reshape(28, 28), cmap='gray')
    true_label = y_test_multi[i]
    pred_label = y_pred_classes[i]
    confidence = y_pred_multi[i][pred_label]
    
    color = 'green' if true_label == pred_label else 'red'
    plt.title(f'T:{true_label} P:{pred_label}\n{confidence:.2%}', 
              fontsize=8, color=color)
    plt.axis('off')
plt.tight_layout()
plt.show()

---

## 4. 多标签分类问题 (Multi-label Classification)

### 关键配置
- **输出层**: N个神经元 (N=标签数) + sigmoid激活
- **损失函数**: `binary_crossentropy` (每个标签独立二分类)
- **标签格式**: k-hot编码 (可以同时为多个类别)

### 与多分类的区别
- 多分类: 互斥类别（softmax），只能属于一个类
- 多标签: 非互斥（sigmoid），可以属于多个类

### 典型应用
- 图片标签预测（一张图可能有多个标签）
- 文本主题分类（一篇文章可能属于多个主题）
- 电影类型分类（动作+冒险+科幻）
- 疾病诊断（患者可能同时患有多种疾病）

In [ ]:
# 生成多标签分类数据集
X_multilabel, y_multilabel = make_multilabel_classification(
    n_samples=5000,
    n_features=50,
    n_classes=10,
    n_labels=3,  # 平均每个样本有3个标签
    random_state=42
)

# 数据划分
X_train_ml, X_test_ml, y_train_ml, y_test_ml = train_test_split(
    X_multilabel, y_multilabel, test_size=0.2, random_state=42
)

# 标准化
scaler_ml = StandardScaler()
X_train_ml = scaler_ml.fit_transform(X_train_ml)
X_test_ml = scaler_ml.transform(X_test_ml)

print(f"训练集形状: {X_train_ml.shape}")
print(f"标签形状: {y_train_ml.shape}")
print(f"标签示例(k-hot): {y_train_ml[0]}")
print(f"平均标签数: {y_train_ml.sum(axis=1).mean():.2f}")

In [ ]:
def build_multilabel_classifier(input_dim, num_labels, hidden_layers=[128, 64]):
    """
    构建多标签分类模型
    
    参数:
        input_dim: 输入特征维度
        num_labels: 标签数量
        hidden_layers: 隐藏层神经元数量列表
    
    返回:
        编译后的模型
    """
    model = models.Sequential(name='Multilabel_Classifier')
    
    model.add(layers.Input(shape=(input_dim,)))
    
    # 隐藏层
    for i, units in enumerate(hidden_layers):
        model.add(layers.Dense(units, activation='relu', name=f'hidden_{i+1}'))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(0.3))
    
    # 输出层：N个神经元 + sigmoid激活
    # 每个输出独立进行二分类判断
    model.add(layers.Dense(num_labels, activation='sigmoid', name='output'))
    
    return model

In [ ]:
# 构建并编译多标签分类模型
model_multilabel = build_multilabel_classifier(
    input_dim=X_train_ml.shape[1],
    num_labels=y_train_ml.shape[1]
)

model_multilabel.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',  # 每个标签独立的二分类损失
    metrics=[
        keras.metrics.BinaryAccuracy(name='accuracy'),
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall')
    ]
)

model_multilabel.summary()

In [ ]:
# 训练模型
history_multilabel = model_multilabel.fit(
    X_train_ml, y_train_ml,
    batch_size=128,
    epochs=30,  # 测试时用2个epoch
    validation_split=0.2,
    verbose=1
)

In [ ]:
# 评估多标签分类模型
results = model_multilabel.evaluate(X_test_ml, y_test_ml, verbose=0)
print("\n" + "="*50)
print("多标签分类模型测试结果:")
for name, value in zip(model_multilabel.metrics_names, results):
    print(f"{name}: {value:.4f}")

# 预测
y_pred_ml = model_multilabel.predict(X_test_ml[:5], verbose=0)
y_pred_binary = (y_pred_ml > 0.5).astype(int)

print("\n预测示例:")
for i in range(5):
    print(f"样本 {i+1}:")
    print(f"  真实标签: {y_test_ml[i]}")
    print(f"  预测标签: {y_pred_binary[i]}")
    print(f"  预测概率: {y_pred_ml[i].round(2)}")
    print()

---

## 5. 回归问题 (Regression)

### 关键配置
- **输出层**: 
  - 单变量回归: 1个神经元，无激活函数
  - 多变量回归: N个神经元，无激活函数
- **损失函数**: 
  - `mean_squared_error` (MSE): 惩罚大误差
  - `mean_absolute_error` (MAE): 对离群点更鲁棒
  - `huber_loss`: MSE和MAE的折衷
- **评估指标**: MSE, MAE, R²

### 典型应用
- 房价预测
- 股票价格预测
- 销量预测
- 年龄估计

In [ ]:
# 生成回归数据集
X_reg, y_reg = make_regression(
    n_samples=5000,
    n_features=30,
    n_informative=20,
    noise=10,
    random_state=42
)

# 数据划分
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)

# 标准化输入和输出
scaler_X_reg = StandardScaler()
scaler_y_reg = StandardScaler()

X_train_reg = scaler_X_reg.fit_transform(X_train_reg)
X_test_reg = scaler_X_reg.transform(X_test_reg)

y_train_reg = scaler_y_reg.fit_transform(y_train_reg.reshape(-1, 1)).flatten()
y_test_reg = scaler_y_reg.transform(y_test_reg.reshape(-1, 1)).flatten()

print(f"训练集形状: {X_train_reg.shape}")
print(f"目标值范围: [{y_train_reg.min():.2f}, {y_train_reg.max():.2f}]")

In [ ]:
def build_regressor(input_dim, output_dim=1, hidden_layers=[128, 64, 32]):
    """
    构建回归模型
    
    参数:
        input_dim: 输入特征维度
        output_dim: 输出维度（单变量=1，多变量=N）
        hidden_layers: 隐藏层神经元数量列表
    
    返回:
        编译后的模型
    """
    model = models.Sequential(name='Regressor')
    
    model.add(layers.Input(shape=(input_dim,)))
    
    # 隐藏层
    for i, units in enumerate(hidden_layers):
        model.add(layers.Dense(units, activation='relu', name=f'hidden_{i+1}'))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(0.2))
    
    # 输出层：无激活函数
    # 输出可以是任意实数
    model.add(layers.Dense(output_dim, name='output'))
    
    return model

In [ ]:
# 构建并编译回归模型
model_regression = build_regressor(input_dim=X_train_reg.shape[1])

model_regression.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='mse',  # 均方误差
    metrics=[
        keras.metrics.MeanAbsoluteError(name='mae'),
        keras.metrics.RootMeanSquaredError(name='rmse')
    ]
)

model_regression.summary()

In [ ]:
# 训练模型
history_regression = model_regression.fit(
    X_train_reg, y_train_reg,
    batch_size=128,
    epochs=30,  # 测试时用2个epoch
    validation_split=0.2,
    verbose=1
)

In [ ]:
# 评估回归模型
results = model_regression.evaluate(X_test_reg, y_test_reg, verbose=0)
print("\n" + "="*50)
print("回归模型测试结果:")
for name, value in zip(model_regression.metrics_names, results):
    print(f"{name}: {value:.4f}")

# 预测并计算R²
y_pred_reg = model_regression.predict(X_test_reg, verbose=0).flatten()
r2 = r2_score(y_test_reg, y_pred_reg)
print(f"R² Score: {r2:.4f}")

# 可视化预测结果
plt.figure(figsize=(12, 5))

# 预测值 vs 真实值散点图
plt.subplot(1, 2, 1)
plt.scatter(y_test_reg, y_pred_reg, alpha=0.5)
plt.plot([y_test_reg.min(), y_test_reg.max()], 
         [y_test_reg.min(), y_test_reg.max()], 'r--', lw=2)
plt.xlabel('真实值')
plt.ylabel('预测值')
plt.title(f'预测值 vs 真实值 (R²={r2:.4f})')
plt.grid(True, alpha=0.3)

# 残差分布
plt.subplot(1, 2, 2)
residuals = y_test_reg - y_pred_reg
plt.hist(residuals, bins=50, edgecolor='black')
plt.xlabel('残差')
plt.ylabel('频数')
plt.title('残差分布')
plt.axvline(x=0, color='r', linestyle='--', lw=2)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## 6. 关键知识总结

### 任务类型对照表

| 任务类型 | 输出层配置 | 激活函数 | 损失函数 | 标签格式 |
|---------|-----------|---------|---------|----------|
| **二分类** | 1个神经元 | sigmoid | binary_crossentropy | 0/1 |
| **多分类** | N个神经元 | softmax | categorical_crossentropy | one-hot |
| **多分类(整数标签)** | N个神经元 | softmax | sparse_categorical_crossentropy | 整数 |
| **多标签** | N个神经元 | sigmoid | binary_crossentropy | k-hot |
| **回归** | 1个神经元 | 无 | mse/mae | 连续值 |
| **多变量回归** | N个神经元 | 无 | mse/mae | 连续向量 |

### Dense层参数计算

对于 `Dense(units)`，如果输入维度为 `D_in`:
- **参数量** = `D_in × units + units` (权重 + 偏置)
- **输出形状** = `(batch_size, units)`

### 网络设计原则

1. **隐藏层数量**
   - 简单问题: 1-2层
   - 中等复杂: 3-4层
   - 复杂问题: 5层以上

2. **神经元数量**
   - 输入层: 根据特征数量
   - 隐藏层: 通常逐层递减 (256 → 128 → 64)
   - 输出层: 根据任务类型

3. **激活函数选择**
   - 隐藏层: ReLU（首选）、LeakyReLU、ELU
   - 输出层: 根据任务类型（见上表）

4. **正则化技术**
   - Dropout: 0.2-0.5
   - Batch Normalization: 加速训练
   - L1/L2正则化: 控制权重大小

### 常见问题与解决方案

| 问题 | 现象 | 解决方案 |
|------|------|----------|
| **过拟合** | 训练精度高，验证精度低 | 增加Dropout、减少神经元、数据增强 |
| **欠拟合** | 训练和验证精度都低 | 增加网络容量、训练更多epoch |
| **梯度消失** | 深层网络训练困难 | 使用ReLU、BatchNorm、ResNet结构 |
| **类别不平衡** | 偏向多数类 | 类权重、重采样、focal loss |
| **训练慢** | 收敛速度慢 | 提高学习率、BatchNorm、优化器调整 |

---

## 完整训练配置模板

```python
# 实际项目中建议使用的完整训练配置

# 回调函数
# callbacks = [
#     keras.callbacks.EarlyStopping(
#         monitor='val_loss',
#         patience=15,
#         restore_best_weights=True
#     ),
#     keras.callbacks.ReduceLROnPlateau(
#         monitor='val_loss',
#         factor=0.5,
#         patience=7,
#         min_lr=1e-7
#     ),
#     keras.callbacks.ModelCheckpoint(
#         'best_model.h5',
#         monitor='val_accuracy',
#         save_best_only=True
#     )
# ]

# 训练
# history = model.fit(
#     X_train, y_train,
#     batch_size=64,
#     epochs=100,
#     validation_split=0.2,
#     callbacks=callbacks,
#     verbose=1
# )
```